# Empirical Project 8

---
**Download the code**

To download the code used in this project as a notebook that can be run in Visual Studio Code, Google Colab, or Jupyter Notebook, right click [here]() and select 'Save Link As', then save it as a `.ipynb` file.

Don’t forget to also download the data into your working directory by following the steps in this project.

---

## Getting started in Python

For this project, you will need the following packages:

- **pandas** for data analysis
- **matplotlib** for data visualisation
- **numpy** for numerical methods
- **statsmodels** for an extra statistics function

You'll also be using the **warnings** and **pathlib** packages, but these come built-in with Python.

Remember, you can install packages in Visual Studio Code's integrated terminal (click "View > Terminal") by running `conda install packagename` (if using the Anaconda distribution of Python) or `pip install packagename` if not.

Once you have the Python packages installed, you will need to import them into your Python session—and configure any other initial settings.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings

# Set the plot style for prettier charts:
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams["figure.dpi"] = 150

# Ignore warnings to make nice output
warnings.simplefilter("ignore")

## Python Walkthrough 8.1

**Importing data into Python**

As we are importing an Excel file, we use the `pd.read_excel` function provided by the **pandas** package. The file is called Project-8-datafile.xlsx and is saved into a subfolder of our working directory called 'data'. The file contains four worksheets that contain data, and these are named ‘Wave 1’ through to ‘Wave 4’. We will load the worksheets one by one and add them to the previous worksheets using the `pd.concat` function, which concatenates (combines) dataframes. 

The final output is called `lifesat_data`.

In [ ]:
list_of_sheetnames = ["Wave " + str(i) for i in range(1, 5)]
list_of_dataframes = [
    pd.read_excel(Path("data/Project-8-datafile.xlsx"), sheet_name=x)
    for x in list_of_sheetnames
]
lifesat_data = pd.concat(list_of_dataframes, axis=0)
lifesat_data.head()

Note that reading in data from lots of Excel worksheets is quite slow. For large datasets, parquet is a very efficient format and works across programming languages.

The variable names provided in the spreadsheet are not very specific (a combination of letters and numbers that don’t tell us what the variable measures). To make it easier to keep track we could two approaches:

1. use a multi-index for our columns; this is an index with more than one entry per column, with multiple column names stacked on top of each other. We would create a multi-index that includes the original codes, then has labels, and then has a short description.

2. Work with what we have, but keep hold of an easy way to convert the codes into either labels or a short description should we need to.


Using a multi-index for columns (option 1) is convenient in some ways but it also has some downsides. The main downside is extra complexity when doing operations on columns because we'll need to specify *all* of the different names of a column in some ways. This is so that there's no ambiguity in the case where the some of the column names are repeated at one level off the multi-index. You can see why this might be needed—you could have a case where some column names are repeated on some levels of the multi-index. So, although using the syntax

```python
lifesat_data["A009"]
```

will work most of the time, for *some* operations we would have to use

```python
lifesat_data[("A009", "Health", "State of health (subjective)")]
```

instead to access the health column. As you can see, we are removing any ambiguity about which data we refer to by specifying all three of its possible names in an object enclosed by curvy brackets (this object is called a `tuple` and behaves a lot like a list, except you can't modify individual values within it).

Option 2 has the downside that on typical dataframe operations, we will only have the codes to go on and will have to look those codes up if we need to remind ourselves of what they represent. The easiest way to do this is using dictionaries.

In this tutorial, we'll go for option 2 as it's simpler, and there's a lot to be said for making life simpler. However, if you do want to go down the option 1 route, you can and the first step would be to create the multi-index column object like so:

```python
# option 1 only
index = pd.MultiIndex.from_tuples(
    tuple(zip(lifesat_data.columns, labels, short_description)),
    names=["code", "label", "description"],
)
lifesat_data.columns = index
```

where `labels` and `short_descriptions` are lists of strings and the zip function turns the three lists of details (codenames, labels, and short descriptions) into a tuple.

Going back to option 2: let's first create our neat mapping of codes into labels and codes into short descriptions using dictionaries


In [ ]:
labels = [
    "EVS-wave",
    "Country/region",
    "Respondent number",
    "Health",
    "Life satisfaction",
    "Work Q1",
    "Work Q2",
    "Work Q3",
    "Work Q4",
    "Work Q5",
    "Sex",
    "Age",
    "Marital status",
    "Number of children",
    "Education",
    "Employment",
    "Monthly household income",
]

short_descriptions = [
    "EVS-wave",
    "Country/region",
    "Original respondent number",
    "State of health (subjective)",
    "Satisfaction with your life",
    "To develop talents you need to have a job",
    "Humiliating to receive money w/o working for it",
    "People who don't work become lazy",
    "Work is a duty towards society",
    "Work comes first even if it means less spare time",
    "Sex",
    "Age",
    "Marital status",
    "How many living children do you have",
    "Educational level (ISCED-code one digit)",
    "Employment status",
    "Monthly household income (x 1,000s PPP euros)",
]

labels_dict = dict(zip(lifesat_data.columns, labels))
descrp_dict = dict(zip(lifesat_data.columns, short_descriptions))

Let's just check these work looking at the example of health again, which has code `"A009"`

In [ ]:
print(labels_dict["A009"])
print(descrp_dict["A009"])

Throughout this project we will refer to the variables using their original names, the codes, but you can see the extra info when you need to by passing those codes into these dictionaries.

## Python Walkthrough 8.2

**Cleaning data and splitting variables**

*Inspect the data and recode missing values*

Python's **pandas** package stores variables as different types depending on the kind of information the variable represents. For categorical data, where, as the name suggests, data is divided into a number of groups, such as country or occupation, the variables can be stored as the `"category"`. Numerical data (numbers that do not represent categories) can be stored as integers, `"int"`, or real numbers, usually `"double"`. There are other datatypes too, for example `"datetime64[ns]"` for datetimes in nano-second increments. Text is of type `"string"`. There's also a 'not quite sure' datatype, `"object"`, which is typically used for data that doesn't clearly fall into a bucket.

However, **pandas** is quite conservative about deciding on data types for you, so you do have to be careful to check the datatypes are what you want when they are read in. The classic example is of numbers being read in as type `"object"`.

The `.info()` method tells us what data types are being used in a **pandas** dataframe:

In [ ]:
lifesat_data.info()

We have a lot of `"object"` columns, so it's clear that a lot of the columns haven't been read in as what they should be.

Looking back at our data, we can see that there are a LOT of `".a"` values and, reading the original data source, it looks like these represent missing values. Let's replace those with the proper missing value indicator, `pd.NA`.

In [ ]:
lifesat_data = lifesat_data.replace(".a", pd.NA)
lifesat_data.head()

This isn't the only way to deal with those pesky `".a"` values. When we read each file in, we could have replaced the value for missing data used in the file, `".a"`, with **pandas** built-in representation of missing numbers. This is achieved via the `na_values=".a"` keyword in the `pd.read_excel` function.

*Recode the life satisfaction variable*

To recode the life satisfaction variable (`"A170"`), we can use a dictionary to map ‘Dissatisfied’ or ‘Satisfied’ into 1 or 10 respectively. This variable was imported as an object column. After changing the text into numerical values, we use the `astype("Int32")` method to convert the variable into a 32-bit integer (these can represent any integer between -$2^{31}$ and $2^{31}$).

In [ ]:
col_satisfaction = "A170"
lifesat_data[col_satisfaction] = (
    lifesat_data[col_satisfaction]
    .replace({"Satisfied": 10, "Dissatisfied": 1})
    .astype("Int32")
)
lifesat_data.info()

*Recode the variable for number of children*

We repeat this process for the variable indicating the number of children (`"X011_01"`).

In [ ]:
col_num_children = "X011_01"

lifesat_data[col_num_children] = (
    lifesat_data[col_num_children].replace({"No children": 0}).astype("Int32")
)

*Replace text with numbers for multiple variables*

When we have to recode multiple variables with the same mapping of text to numerical value, we can take a bit of a short-cut to recode multiple columns at once.

In [ ]:
col_codes = ["C036", "C037", "C038", "C039", "C041"]

lifesat_data[col_codes] = (
    lifesat_data[col_codes]
    .replace(
        {
            "Strongly disagree": 1,
            "Disagree": 2,
            "Neither agree nor disagree": 3,
            "Agree": 4,
            "Strongly agree": 5,
        }
    )
    .astype("Int32")
)

# This one needs a different mapping

health_code = "A009"
lifesat_data[health_code] = (
    lifesat_data[health_code]
    .replace({"Very poor": 1, "Poor": 2, "Fair": 3, "Good": 4, "Very good": 5})
    .astype("Int32")
)

*Split a variable containing numbers and text*

To split the education variable `"X025A"` into two new columns, we use the `.explode` method, which creates two new variables called Education_1 and Education_2 containing the numeric value and the text description respectively. Then we use the mutate_at function to convert Education_1 into a numeric variable.

Because we're still using a multi-layered column system, we'll need to specify precise which combination of column names we're using in a tuple (as we just did above):

In [ ]:
education_code = "X025A"
lifesat_data[education_code].str.split(" : ", expand=True)

Let's do this again but save it back into our dataframe under two new column names. We'll pass these back in a list.

In [ ]:
ed_num, ed_sch = [education_code + suffix for suffix in ["_num", "_sch"]]

print(ed_num)
print(ed_sch)

Now pass them back in as a list (note the extra square brackets) so that they map up to the two new columns on the right hand side.

In [ ]:
lifesat_data[[ed_num, ed_sch]] = lifesat_data[education_code].str.split(
    " : ", expand=True
)
lifesat_data[ed_num] = pd.to_numeric(lifesat_data[ed_num]).astype("Int32")
lifesat_data.sample(5, random_state=4)

You can see the two extra columns for education at the end of the dataframe.

There's just one more column to convert: monthly income, which is a real number rather than an integer. Let's do that, and then let's have a final look at our object types:

In [ ]:
lifesat_data["X047D"] = pd.to_numeric(lifesat_data["X047D"])
lifesat_data.info()

## Python Walkthrough 8.3

**Dropping specific observations**

As not all questions were asked in all waves, we have to be careful when dropping observations with missing values for certain questions, to avoid accidentally dropping an entire wave of data. For example, information on self-reported health (`"A009"`) was not recorded in Wave 3, and questions on work attitudes (`"C036"` to `"C041"`) and information on household income are only asked in Waves 3 and 4. Furthermore, information on the number of children (`"X011_01"`) and education (`"X025A"`) are only collected in the final wave.

We will first use the `.dropna()` function to find only those observations with complete information on variables present in all waves (`"X003"`, `"A170"`, `"X028"`, `"X007"`, and `"X001"`, which we will store in a list named `include`). Combining with `.index` will enable us to find the index values for rows that have complete information. But we must also be wary that our index is currently not unique, so we'll do a reset of the index first to ensure that there is one and only one index value for each observation (this is generally good practice!) using `.reset_index()` with the keyword argument `drop=True` because we don't wish to keep the current index in the dataframe.

In [ ]:
include = ["X003", "A170", "X028", "X007", "X001"]
lifesat_data = lifesat_data.reset_index(drop=True)
lifesat_data = lifesat_data.loc[lifesat_data[include].dropna().index, :]

Next we will look at variables that were only present in some waves. For each variable/group of variables, we have to only look at the particular wave(s) in which the question was asked, then keep the observations with complete information on those variables. As before, we make lists of variables that only feature in Waves 1, 2, and 4 (`"A009"`—stored in `include_wave_1_2_4`), Waves 3 and 4 (`"C036"` to `"C041"`, `"X047D"`—stored in `include_wave_3_4`), and Wave 4 only (`"X011_01"` and `"X025A"`—stored in `include_wave_4`). Again we will use the `.dropna()` method, but combine it with the logical OR operator, `|`, to include all observations for waves that did not ask that question, along with the complete cases for that question in the other waves.

First, we put together some useful background info on what questions were only included in which waves.

In [ ]:
# A009 is not in Wave 3.
# Note that even though it's just one entry, we use square brackets to make it a list
include_in_wave_1_2_4 = ["A009"]
# Work attitudes and income are in Waves 3 and 4.
include_in_wave_3_4 = ["C036", "C037", "C038", "C039", "C041", "X047D"]
# Number of children and education are in Wave 4.
include_in_wave_4 = ["X011_01", "X025A"]

Now we check the cases for these waves, successively refining the data to just those with ...

For example, in the first refinement of the data below we will first pick out any row for which the column `"A009"` has an entry *or* (represented by `|`) `"S002EVS"` takes the values `"1981-1984"` or `"1990-1993"`. This keeps observations if they are in Wave 1 (1981-1984) or Wave 2 (1990-1993), or they are in Waves 3 or 4 with complete information. Because this will create two columns worth of boolean values, we will then use the row-wise (`axis=1`) `.any()` method to create a single boolean value, `True` or `False`, for every row. Then we will use those boolean values to filter down the `lifesat_data` to only the rows that satisfy the given condition.

In [ ]:
# in Wave 1 (1981-1984) or Wave 2 (1990-1993), or they are in Waves 3 or 4 with complete information
condition_wave_3_4 = (lifesat_data[include_in_wave_3_4].notna()).all(axis=1) | (
    lifesat_data["S002EVS"].isin(["1981-1984", "1990-1993"])
)
lifesat_data = lifesat_data.loc[condition_wave_3_4, :]

# in Wave 4 with complete information on the questions specific to that wave or not in Wave 4
condition_wave_4 = (
    lifesat_data[include_in_wave_4].notna().all(axis=1)
) | ~lifesat_data["S002EVS"].isin(["2008-2010"])
lifesat_data = lifesat_data.loc[condition_wave_4, :]

# in Waves 1, 2, or 4 with complete information on the questions specific to those waves, or in Wave 3
condition_wave_1_2_4 = (lifesat_data[include_in_wave_1_2_4].notna().all(axis=1)) | (
    lifesat_data["S002EVS"].isin(["1999-2001"])
)
lifesat_data = lifesat_data.loc[condition_wave_1_2_4, :]

## Python Walkthrough 8.4

**Calculating averages and percentiles**

*Calculate average work ethic score*

We use the `.mean(axis=1)` (remember it's `axis=0` to aggregate over index and `axis=1` to aggregrate over columns) method to calculate the average work ethic score for each observation (`"workethic"`) based on the five survey questions related to working attitudes (`"C036"` to `"C041"`). As we're still using a multi-level column naming convention, we need to specify three levels of column names to create a new column—but they can all be the same.

In [ ]:
lifesat_data["work_ethic"] = lifesat_data.loc[
    :, ["C036", "C037", "C038", "C039", "C041"]
].mean(axis=1)
lifesat_data.sample(5, random_state=5)

**Calculating averages and percentiles**

Regression package **statsmodels** provides a handy method (`"ECDF"`) to obtain an individual’s relative income as a percentile. As we want a separate income distribution for each wave, we use the `.groupby` method. Finally, we tidy up the results by converting each value into a percentage and rounding to a single decimal place.

The ecdf function will not work if there is missing data. Since we don’t have income data for Waves 1 and 2, we will have to split the data when working out the percentiles. First we store all observations from Waves 3 and 4 separately in a temporary dataset (df.new), only keep observations that have income values (!is.na(X047D)), calculate the percentile values with the ecdf function, and save the values in a variable called percentile (which will be ‘NA’ for Waves 1 and 2). Then we recombine this data with the original observations from Waves 1 and 2.


In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

# create empty col for new variable
lifesat_data["inc_percentile"] = pd.NA

# fill it for waves 3 and 4 with relevant data
condition_inc_percentile = (
    lifesat_data["S002EVS"].isin(["1999-2001", "2008-2010"])
) & (lifesat_data["X047D"].notna())

lifesat_data.loc[condition_inc_percentile, "inc_percentile"] = (
    lifesat_data.loc[
        condition_inc_percentile, :
    ]  # Select waves 3 & 4 without missing income data
    .groupby("S002EVS")["X047D"]  # groupby wave  # select income variable
    .transform(
        lambda x: np.round(ECDF(x)(x) * 100, 1)
    )  # compute ecdf as % round to 1 decimal place
)

# see the dataframe with the new column
lifesat_data.sample(5, random_state=5)

## Python Walkthrough 8.5

**Calculating summary statistics**

*Create a table showing employment status, by country*

One of the most useful features of **pandas** is its composability. We can stack up multiple methods to create just the statistics we want. In this example, we're going to use a succession of methods to create a table showing the employment status (as a percentage) of each country's labour force. The steps are:

- Select the data for Wave 4
- Group it by employment type (`"X028"`) and country (`"S003"`). Order will matter later when we use `unstack`; whichever variable is last in the groupby command will be switched from the index to the columns when we call unstack.
- Select the column to take observations from. In this case, it makes sense to use employment again.
- Count the number of observations
- Unstack so that we have a table instead of a long list (with countries as columns)
- Transform the numbers into percentages that sum to 100 for each country
- Round the values in the table
- Because we have more countries than employment statuses, transpose the columns and index

In [ ]:
sum_table = (
    lifesat_data.loc[
        lifesat_data["S002EVS"] == "2008-2010", :
    ]  # Wave 4 only, all columns
    .groupby(["X028", "S003"])[  # Group by employment and country
        "X028"
    ]  # Select employment column
    .count()  # Count number of observations in each category
    .unstack()  # Turn countries from an index into columns (countries as last groupby variable)
    .transform(lambda x: x * 100 / x.sum())  # Compute a percentage
    .round(2)  # Round to 2 decimal places
    .T  # Tranpose so countries are the index, employment types the columns
)

sum_table

If we then wanted to export this table for further use elsewhere, we would export it with `sum_table.to_html(filename)`, `sum_table.to_excel(filename)`, `sum_table.to_string(filename)`, `sum_table.to_latex(filename)`, or many other options that you can find [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html).

*Calculate summary statistics by gender*

We can also obtain summary statistics on a number of variables at the same time using the `apply` function. To obtain the mean value for each of the required variables, grouped by the gender variable (`"X001"`), we can compose methods again:

In [ ]:
(
    lifesat_data.loc[
        lifesat_data["S002EVS"] == "2008-2010", :
    ]  # Wave 4 only, all columns
    .groupby(["X001"])[  # Group by gender
        "A009", "A170", "work_ethic", "X003", "X025A_num", "X011_01"
    ]  # Select columns
    .mean()
    .round(2)  # Round to 2 decimal places
)

Getting the standard deviation is as simple as replacing `mean()` with `std()`

In [ ]:
(
    lifesat_data.loc[
        lifesat_data["S002EVS"] == "2008-2010", :
    ]  # Wave 4 only, all columns
    .groupby(["X001"])[  # Group by gender
        "A009", "A170", "work_ethic", "X003", "X025A_num", "X011_01"
    ]  # Select columns
    .std()
    .round(2)  # Round to 2 decimal places
)

But what if we want *both*!? We can have that too, using the `agg` (short for aggregate) method and a list of functions.

In [ ]:
(
    lifesat_data.loc[
        lifesat_data["S002EVS"] == "2008-2010", :
    ]  # Wave 4 only, all columns
    .groupby(["X001"])[  # Group by gender
        "A009", "A170", "work_ethic", "X003", "X025A_num", "X011_01"
    ]  # Select columns
    .agg(["mean", "std"])
    .round(2)  # Round to 2 decimal places
)

Note that, in this case, we have a multi-level column object in our dataframe. If you want to flip down the last level of columns to be an index, try using `.stack()`.

If you're exporting your results, you're not going to want to use the code names though. So you'll probably want to export your table with the nice names substituted in. You can do this using the dictionaries we created right at the start. Let's use the `label_dict` with a stacked version of the table above.

In [ ]:
tab = (
    lifesat_data.loc[
        lifesat_data["S002EVS"] == "2008-2010", :
    ]  # Wave 4 only, all columns
    .groupby(["X001"])[  # Group by gender
        "A009", "A170", "work_ethic", "X003", "X025A_num", "X011_01"
    ]  # Select columns
    .agg(["mean", "std"])
    .round(2)  # Round to 2 decimal places
    .stack()  # bring mean and std into the index
    .rename(labels_dict, axis=1)  # rename the columns
)

tab

We didn't rename `"work_ethic"` and `"X025A_num"`, because we created those variables *after* the labels dictionary was created (so there are no labels for them). And `"X001"` is not a column heading, it's the index name, so we'll have to change that separately.

Let's update our dictionary with some new names and a new index name.

In [ ]:
labels_dict.update({"work_ethic": "Work Ethic", "X025A_num": "Education Level"})
tab = tab.rename(labels_dict, axis=1)
tab.index.names = [
    "",
    "",
]  # Set index names empty (two levels because two column levels)

tab